In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
cd drive/MyDrive/10707/src

/content/drive/.shortcut-targets-by-id/1mLIpKobxLqDPVzGz-XXAA4JO-lEjQbax/10707/src


In [5]:
## Import Libraries

import models
from models import *

import dataset
from dataset import *

import optimizer
from optimizer import *

import utils 
from utils import *

import train 
from train import *

import pickle

In [ ]:
## Import Data

with open("data/EEG_Moabb.pickle","rb") as f:
  data_for_EEG = pickle.load(f)

In [ ]:
data_for_EEG[1][0].shape

(90, 64, 481)

In [ ]:
## Build Dataset

dataset = EEGBiometricDataset(list(range(1,40)),data_for_EEG)
n_classes = dataset.num_subjects()

In [ ]:
## Split Into Train and Test

train_dataset, test_dataset = create_train_test(dataset,split = 0.8)
train_loader, test_loader = create_loaders(train_dataset,test_dataset,batch_size=32)

In [7]:
## Initiliaze Model

# model = EEGFusionNet()
n_classes = 39
model = EEGBiometricNet(n_classes)
model.cuda()

EEGBiometricNet(
  (conv1): Conv2d(1, 16, kernel_size=(1, 64), stride=(1, 1))
  (avgpool1): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(32, 2), stride=(1, 1))
  (avgpool2): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=1056, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=39, bias=True)
)

In [9]:
import torchsummary
from torchsummary import summary

In [11]:
summary(model,(1,64,160))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 97]           1,040
         AvgPool2d-2           [-1, 16, 64, 12]               0
       BatchNorm2d-3           [-1, 16, 64, 12]              32
            Conv2d-4           [-1, 32, 33, 11]          32,800
         AvgPool2d-5            [-1, 32, 33, 1]               0
       BatchNorm2d-6            [-1, 32, 33, 1]              64
            Linear-7                   [-1, 64]          67,648
            Linear-8                   [-1, 39]           2,535
Total params: 104,119
Trainable params: 104,119
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 1.05
Params size (MB): 0.40
Estimated Total Size (MB): 1.49
----------------------------------------------------------------


In [ ]:
## Create Optimizer

optimizer = build_optimizer(model,'adam',3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=5,factor=0.5)

In [ ]:
model,optimizer = train(model,optimizer,train_loader,test_loader,num_epochs=100,scheduler=scheduler)

Train Dataset 8424
Test Dataset 2106

 Model Name :  EEGFusionNet 



RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), "saved_models/BiometricNet.pth")